## Welcome to the Second Lab - Week 1, Day 3

Today we will work with lots of models! This is a way to get comfortable with APIs.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Important point - please read</h2>
            <span style="color:#ff7800;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations.<br/><br/>If you have time, I'd love it if you submit a PR for changes in the community_contributions folder - instructions in the resources. Also, if you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...
            </span>
        </td>
    </tr>
</table>

In [1]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [2]:
# Always remember to do this!
load_dotenv(override=True)

True

In [3]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key not set (and this is optional)


In [4]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [5]:
messages

[{'role': 'user',
  'content': 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question, no explanation.'}]

In [6]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
question = response.choices[0].message.content
print(question)


How would you approach resolving a moral dilemma where the utilitarian outcome benefits the majority but causes significant harm to a vulnerable minority, and what ethical principles would guide your decision-making process?


In [7]:
competitors = []
answers = []
messages = [{"role": "user", "content": question}]

In [8]:
# The API we know well

model_name = "gpt-4o-mini"

response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Resolving a moral dilemma that pits utilitarian outcomes against the rights and well-being of a vulnerable minority requires a careful and nuanced approach. Here are the steps I would consider, along with the ethical principles that would guide my decision-making:

1. **Identify the Stakeholders**: Begin by clearly identifying all stakeholders involved, including the majority that would benefit from the utilitarian outcome and the vulnerable minority that would be harmed. Understanding their circumstances and needs is crucial.

2. **Consider the Consequences**: Utilize a utilitarian framework to assess the overall consequences of the decision. This means evaluating the benefits to the majority and the harms to the minority. However, it's important to go beyond mere numbers and consider the nature and severity of the harm to the minority.

3. **Examine Ethical Principles**:
   - **Rights-Based Ethics**: Consider the rights of the vulnerable minority. A rights-based approach emphasizes that certain rights (such as the right to life, liberty, and security) should not be violated, even for the greater good. If the harm to the minority is severe, it may override utilitarian calculations.
   - **Justice and Fairness**: Reflect on principles of justice and fairness. Are the burdens and benefits distributed equitably? A fair distribution of harms and benefits is essential to ethical decision-making.
   - **Care Ethics**: Consider the relationships involved and the importance of care and compassion for the vulnerable. This perspective emphasizes empathy and the moral imperative to support those who are most at risk.

4. **Alternative Solutions**: Look for alternative approaches that could achieve the desired outcomes for the majority without causing significant harm to the minority. This might include compromise solutions or interventions that provide support to the vulnerable group.

5. **Seek Input and Dialogue**: Engage with the affected groups, particularly the vulnerable minority, to understand their perspectives and insights. This can lead to more informed and inclusive decision-making.

6. **Make a Decision with Transparency**: After careful consideration of all factors and principles, make a decision and be open about the reasoning behind it. Transparency fosters trust and accountability.

7. **Monitor and Adjust**: If the decision is implemented, monitor its outcomes and be ready to make adjustments if the impacts on the vulnerable minority are more harmful than anticipated or if unforeseen consequences arise.

In summary, the decision-making process should balance utilitarian principles with an acknowledgment of rights, justice, compassion, and the voices of those affected. It is vital to act thoughtfully and consider every angle to foster a more ethical and inclusive outcome.

In [9]:
# Anthropic has a slightly different API, and Max Tokens is required

model_name = "claude-3-7-sonnet-latest"

claude = Anthropic()
response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
answer = response.content[0].text

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

# Approaching the Moral Dilemma

This is a classic ethical tension that challenges pure utilitarian thinking. I'd approach it through multiple ethical frameworks:

First, I'd examine if the utilitarian calculation itself is complete. Does it account for:
- Long-term psychological impacts on society from normalizing harm to minorities
- Precedent-setting effects
- Erosion of rights protections

I'd particularly consider:

**Rights-based principles**: Are fundamental rights of the minority being violated? Rights typically function as constraints on utilitarian calculations.

**Justice and fairness**: Would the distribution of benefits and harms reinforce existing inequities?

**Care ethics**: How does the solution affect our relationships and obligations to the vulnerable?

In my decision-making, I'd seek solutions that:
1. Preserve core rights while maximizing welfare
2. Distribute unavoidable harms equitably
3. Include compensatory measures if harm cannot be avoided
4. Engage the affected minority in the decision process

Rather than simply accepting the majority-benefit outcome, I'd work to find creative alternatives that achieve the utilitarian goal while better protecting the vulnerable.

In [11]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

This is a classic and incredibly difficult ethical dilemma, highlighting the inherent tension between utilitarianism and other ethical principles like rights-based ethics and justice. Here's how I would approach resolving it, along with the guiding principles:

**1. Understanding the Situation Thoroughly:**

*   **Fact-Finding:**  I would start by gathering as much detailed information as possible.  This includes:
    *   **Specifics of the benefit to the majority:**  How large is the benefit?  How many people are affected?  Is it a critical benefit (e.g., saving lives) or a more minor one (e.g., increased convenience)?  Are there alternative ways to achieve the benefit, even if they are less efficient?
    *   **Specifics of the harm to the minority:** How severe is the harm? Is it physical, emotional, economic, or a combination? How many people are affected? Is the harm reversible? Is the minority group already disadvantaged or vulnerable?
    *   **Long-term consequences:**  What are the potential long-term effects of both the benefit to the majority and the harm to the minority? Will the harm to the minority create a precedent that could lead to future injustices? Will the benefit to the majority create dependencies or unintended negative consequences?
    *   **Stakeholder perspectives:**  I would seek to understand the perspectives of all stakeholders, including those in the majority, the vulnerable minority, and any neutral parties who might be affected.

*   **Clarifying Values:**  I would explicitly acknowledge the conflicting values at play:
    *   **Maximizing overall happiness (Utilitarianism):**  Seeking the greatest good for the greatest number.
    *   **Protecting individual rights:** Ensuring that every individual, especially the vulnerable, is treated with dignity and respect.
    *   **Promoting justice and fairness:**  Ensuring that burdens and benefits are distributed equitably.
    *   **Minimizing harm:** Avoiding actions that cause unnecessary suffering.

**2. Exploring Alternatives:**

The goal is to find a solution that minimizes harm to the minority while still providing as much benefit as possible to the majority. This involves brainstorming and evaluating different approaches:

*   **Mitigation Strategies:**  Can the harm to the minority be mitigated or minimized?  For example, can compensation be provided, support services offered, or protections put in place to lessen the negative impact?
*   **Benefit Sharing:** Can some of the benefits enjoyed by the majority be shared with the minority to offset the harm they are experiencing?
*   **Targeted Assistance:** Can specific assistance be provided to the minority to help them adapt to the changes and improve their situation?
*   **Finding Alternative Solutions:**  Are there alternative solutions that would achieve the desired outcome for the majority without causing harm to the minority, even if they are less efficient or more costly?  Think creatively.  Can technology help?  Can policy changes help?  Can a different approach altogether be considered?
*   **Phased Implementation:**  Could the solution be implemented in phases, allowing time to monitor the impact on the minority and adjust the approach as needed?

**3. Ethical Frameworks for Decision-Making:**

I would use a combination of ethical frameworks to guide my decision, rather than relying solely on utilitarianism:

*   **Utilitarianism (as a starting point):**  Weigh the overall benefits to the majority against the harm to the minority.  Quantify the impacts as much as possible (though recognizing the limitations of quantifying subjective experiences).  Consider long-term consequences.  However, utilitarianism alone is insufficient.

*   **Deontology (Rights-Based Ethics):**  This framework emphasizes moral duties and rights.  Consider whether the proposed action violates any fundamental rights of the minority, such as the right to life, liberty, or security.  It focuses on the *intrinsic* wrongness of an action, regardless of its consequences.  Are we using the minority as a mere means to an end?

*   **Justice and Fairness:**  Consider the principles of distributive justice.  Is the burden being placed on the minority disproportionate? Is the decision fair and equitable to all affected parties? Are the vulnerable members of society being treated with special consideration?  John Rawls's "veil of ignorance" thought experiment can be helpful here: imagine you don't know which group you'll belong to – would you still find the decision acceptable?

*   **Virtue Ethics:**  What would a virtuous person do in this situation?  What virtues are relevant, such as compassion, fairness, integrity, and responsibility?  How can these virtues guide the decision-making process?

*   **The "Moral Minimum":**  At an absolute minimum, ensure that the proposed solution does not violate basic human rights and provides a level of protection to the minority that respects their dignity and worth.

**4. Making the Decision:**

*   **Balancing Competing Values:**  The decision-making process requires a careful balancing of competing values.  There is no easy answer, and the "right" decision will often depend on the specific circumstances.

*   **Transparency and Accountability:**  The decision-making process should be transparent and accountable.  The rationale for the decision should be clearly explained to all stakeholders, and there should be mechanisms in place to ensure that the decision is implemented fairly and effectively.

*   **Prioritizing the Vulnerable:**  When in doubt, prioritize the well-being of the vulnerable minority.  A just society is judged by how it treats its most vulnerable members.

*   **Ongoing Monitoring and Evaluation:**  After the decision is implemented, it is crucial to monitor its impact on all stakeholders, especially the minority.  If the harm to the minority is greater than anticipated, the decision should be re-evaluated and adjusted accordingly.

**Example Scenario:**

Imagine a city needs to build a new highway to alleviate traffic congestion.  The proposed route goes through a low-income neighborhood with a predominantly minority population, displacing many residents and potentially increasing pollution in the area.

Applying the above framework, I would:

1.  **Investigate:**  Research alternative routes, mitigation strategies (e.g., noise barriers, pollution reduction measures), and compensation packages for displaced residents.
2.  **Evaluate:**  Assess the economic benefits of the highway versus the social and environmental costs to the neighborhood.
3.  **Consider Ethics:**  Recognize the potential violation of the residents' right to housing and a healthy environment.  Explore whether the proposed action perpetuates existing inequalities.
4.  **Seek Alternatives:**  Consider public transportation improvements, traffic management strategies, or a different highway route, even if more expensive.
5.  **Implement:** If the highway is deemed necessary, implement it with maximum mitigation efforts, generous compensation, and community engagement to ensure the residents' voices are heard. Continuously monitor and address negative impacts.

**Conclusion:**

Resolving these dilemmas is rarely simple.  It requires careful analysis, thoughtful consideration of ethical principles, a willingness to explore alternatives, and a commitment to minimizing harm to the most vulnerable.  The goal should not be simply to maximize overall happiness, but to create a just and equitable society where the rights and well-being of all individuals are respected.


In [13]:
deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-chat"

response = deepseek.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Resolving a moral dilemma where utilitarian outcomes conflict with the rights of a vulnerable minority requires a careful balancing of ethical principles. Here’s a structured approach to navigating such a dilemma:

### **1. Clarify the Situation**
   - **Identify Stakeholders:** Who benefits, and who is harmed? How significant are the benefits and harms?
   - **Assess Consequences:** What are the short-term and long-term effects on both the majority and the minority?
   - **Examine Alternatives:** Are there other solutions that could maximize well-being without harming the minority?

### **2. Apply Multiple Ethical Frameworks**
   - **Utilitarianism:** Does the action truly maximize overall happiness, or are there hidden costs (e.g., erosion of trust, systemic injustice)?
   - **Deontological Ethics:** Are fundamental rights (e.g., autonomy, dignity) being violated, regardless of outcomes?
   - **Virtue Ethics:** What would a just, compassionate, and wise person do in this situation?
   - **Justice & Fairness (Rawlsian Ethics):** Would this decision be acceptable behind a "veil of ignorance" (not knowing if you'd be in the majority or minority)?

### **3. Prioritize Protection of the Vulnerable**
   - **Principle of Least Harm:** Even if the majority benefits, is the harm to the minority avoidable or disproportionate?
   - **Rights-Based Approach:** Certain rights (e.g., freedom from exploitation, equal treatment) may be non-negotiable, even for greater utility.
   - **Social Contract Theory:** Does the decision uphold societal trust and fairness, or does it set a dangerous precedent for marginalizing groups?

### **4. Seek a Compromise or Alternative Solution**
   - **Mitigate Harm:** Can safeguards or reparations be implemented to reduce harm to the minority?
   - **Deliberative Democracy:** Engage affected parties in dialogue to find a more just solution.
   - **Long-Term Perspective:** Could short-term gains lead to long-term societal harm (e.g., increased inequality, loss of solidarity)?

### **5. Make a Decision with Ethical Justification**
   - If no perfect solution exists, choose the option that:
     - Minimizes harm to the most vulnerable.
     - Respects fundamental rights.
     - Maintains transparency and accountability.
   - Be prepared to defend the decision based on ethical reasoning, not just outcomes.

### **Key Guiding Principles**
   - **Non-Exploitation:** Avoid sacrificing the few for the many in a way that treats them as mere means.
   - **Proportionality:** The harm to the minority should not vastly outweigh the benefits to the majority.
   - **Moral Integrity:** Upholding justice and dignity may sometimes require rejecting pure utilitarianism.

### **Conclusion**
While utilitarianism provides a useful lens, ethical decision-making should integrate justice, rights, and compassion—especially when vulnerable groups are at risk. The best resolution often lies in finding a middle path that respects all stakeholders rather than accepting a harmful trade-off.

In [ ]:
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)


## For the next cell, we will use Ollama

Ollama runs a local web service that gives an OpenAI compatible endpoint,  
and runs models locally using high performance C++ code.

If you don't have Ollama, install it here by visiting https://ollama.com then pressing Download and following the instructions.

After it's installed, you should be able to visit here: http://localhost:11434 and see the message "Ollama is running"

You might need to restart Cursor (and maybe reboot). Then open a Terminal (control+\`) and run `ollama serve`

Useful Ollama commands (run these in the terminal, or with an exclamation mark in this notebook):

`ollama pull <model_name>` downloads a model locally  
`ollama ls` lists all the models you've downloaded  
`ollama rm <model_name>` deletes the specified model from your downloads

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Super important - ignore me at your peril!</h2>
            <span style="color:#ff7800;">The model called <b>llama3.3</b> is FAR too large for home computers - it's not intended for personal computing and will consume all your resources! Stick with the nicely sized <b>llama3.2</b> or <b>llama3.2:1b</b> and if you want larger, try llama3.1 or smaller variants of Qwen, Gemma, Phi or DeepSeek. See the <A href="https://ollama.com/models">the Ollama models page</a> for a full list of models and sizes.
            </span>
        </td>
    </tr>
</table>

In [ ]:
!ollama pull llama3.2

In [ ]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2"

response = ollama.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
# So where are we?

print(competitors)
print(answers)


In [ ]:
# It's nice to know how to use "zip"
for competitor, answer in zip(competitors, answers):
    print(f"Competitor: {competitor}\n\n{answer}")


In [20]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [ ]:
print(together)

In [22]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [ ]:
print(judge)

In [29]:
judge_messages = [{"role": "user", "content": judge}]

In [ ]:
# Judgement time!

openai = OpenAI()
response = openai.chat.completions.create(
    model="o3-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)


In [ ]:
# OK let's turn this into results!

results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Which pattern(s) did this use? Try updating this to add another Agentic design pattern.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">These kinds of patterns - to send a task to multiple models, and evaluate results,
            are common where you need to improve the quality of your LLM response. This approach can be universally applied
            to business projects where accuracy is critical.
            </span>
        </td>
    </tr>
</table>